In [3]:
import json
import os
from pathlib import Path
from PIL import Image

#클래스 이름을 ID로 매핑
class_map = {
    "집전체":0,
    "지붕":1,
    "집벽":2,
    "문":3, #유무
    "창문":4, #유무
    "굴뚝":5, #유무
    "연기":6, #유무
    "울타리":7, #유무 
    #"길":8,
    #"연못":9,
    "산":8, #유무
    "나무":9, #유무
    "꽃":10, #유무
    "잔디":11, #유무
    "태양":12, #유무

    "나무전체":13,
    "기둥":14, #크기
    "수관":15, 
    "가지":16, #크기, 유무
    "뿌리":17, #유무, 크기
    "나뭇잎":18, #크기
    #"꽃":21,
    "열매":19, #유무
    #"그네":23,
    #"새":24,
    #"다람쥐":25,
    #"구름":26,
    #"달":27,
    #"별":28,

    "사람전체":20,
    "머리":21, #크기
    "얼굴":22, 
    "눈":23,
    "코":24,
    "입":25,
    "귀":26,
    "머리카락":27,
    "목":28,
    "상체":29,
    "팔":30,
    "손":31,
    "다리":32,
    "발":33,
    "단추":34,
    "주머니":35,
    #"운동화":45,
    #"구두":46
}

#YOLO 형식으로 변환
def convert_to_yolo_format(json_data,img_path):
    path=json_data["meta"]["label_path"]
    jpg_path = Path(path).with_suffix(".jpg")
    img_path = img_path+"/"+str(jpg_path)
    im = Image.open(img_path)
    img_width, img_height = im.size

    #YOLO 형식으로 바운딩 박스와 클래스를 저장할 리스트
    yolo_annotations = []

    #바운딩 박스 변환
    for bbox in json_data["annotations"]["bbox"]:
        label = bbox["label"]
        class_id = class_map.get(label, None)
        if class_id is None:
            continue  #클래스가 정의되지 않으면 건너뛰기

        #x, y, w, h -> YOLO 포맷으로 변환
        x_center = (bbox["x"] + bbox["w"] / 2) / img_width
        y_center = (bbox["y"] + bbox["h"] / 2) / img_height
        width = bbox["w"] / img_width
        height = bbox["h"] / img_height

        yolo_annotations.append(f"{class_id} {x_center} {y_center} {width} {height}")

    return yolo_annotations

#YOLO 레이블을 파일로 저장
def save_yolo_labels(json_data, yolo_annotations, save_dir):
    image_name = Path(json_data["meta"]["label_path"]).stem
    label_file = save_dir / f"{image_name}.txt"

    with open(label_file, "w") as f:
        for annotation in yolo_annotations:
            f.write(annotation + "\n")

#JSON 파일 YOLO 형식으로 변환
def convert_json_to_yolo(json_file, save_dir,img_path):
    with open(json_file, "r", encoding="utf-8") as f:
        json_data = json.load(f)

    yolo_annotations = convert_to_yolo_format(json_data, img_path)
    save_yolo_labels(json_data, yolo_annotations, save_dir)

In [ ]:
#JSON 파일을 YOLO 학습을 위해 변환_validation
from tqdm import tqdm

#저장 디렉토리 설정
save_dir = Path(r"C:\Users\User\HTP_Test\YOLO\valid\labels")
save_dir.mkdir(parents=True, exist_ok=True)

folder_path = r"C:\Users\User\HTP_Test\unzipped_data\validation\label"
img_path=r"C:\Users\User\HTP_Test\unzipped_data\validation\image"

#변환 수행
'''
for file in tqdm(os.listdir(folder_path)):
    file_path = os.path.join(folder_path, file)
    convert_json_to_yolo(file_path, save_dir, img_path)
'''
print(len(os.listdir(r"C:\Users\User\HTP_Test\YOLO\valid\labels")))

100%|██████████| 5600/5600 [00:11<00:00, 503.31it/s]


5600


In [17]:
#JSON 파일을 YOLO 학습을 위해 변환_train

#저장 디렉토리 설정
save_dir = Path(r"C:\Users\User\HTP_Test\YOLO\train\labels")
save_dir.mkdir(parents=True, exist_ok=True)

folder_path = r"C:\Users\User\HTP_Test\unzipped_data\train\label"
img_path=r"C:\Users\User\HTP_Test\unzipped_data\train\image"

#변환 수행
'''
for file in tqdm(os.listdir(folder_path)):
    file_path = os.path.join(folder_path, file)
    convert_json_to_yolo(file_path, save_dir, img_path)
'''
print(len(os.listdir(r"C:\Users\User\HTP_Test\YOLO\train\labels")))

100%|██████████| 44800/44800 [01:30<00:00, 497.06it/s]


44800


In [1]:
#이미지 전처리

import cv2
from tqdm import tqdm
import os
import numpy as np
import matplotlib.pyplot as plt

def otsu(folder_path, output_path):
    for file in tqdm(os.listdir(folder_path)):
        img_path=os.path.join(folder_path, file)

        stream = open(img_path, 'rb')
        bytes_array = bytearray(stream.read())
        np_array = np.asarray(bytes_array, dtype=np.uint8)
        img = cv2.imdecode(np_array, cv2.IMREAD_GRAYSCALE)

        #img=cv2.imread(img_path, 0) #그레이스케일 이미지
        clahe = cv2.createCLAHE(clipLimit=1.0, tileGridSize=(8, 8))
        enhanced = clahe.apply(img)
        _, th3=cv2.threshold(enhanced, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)


        filename = os.path.splitext(file)[0]
        save_path = os.path.join(output_path, f"{filename}.jpg")
        ext = os.path.splitext(save_path)[1]
        result, encoded_img = cv2.imencode(ext, th3)
        with open(save_path, mode='wb') as file:
            file.write(encoded_img)

In [2]:
def otsu1(img_path, output_path):
    stream = open(img_path, 'rb')
    bytes_array = bytearray(stream.read())
    np_array = np.asarray(bytes_array, dtype=np.uint8)
    img = cv2.imdecode(np_array, cv2.IMREAD_GRAYSCALE)
    #가우시안 블러
    img = cv2.GaussianBlur(img, (5, 5), 0)

    #CLAHE
    clahe = cv2.createCLAHE(clipLimit=1.0, tileGridSize=(8, 8))
    img = clahe.apply(img)
    
    #Adaptive Thresholding
    img = cv2.adaptiveThreshold(img, 255,  cv2.ADAPTIVE_THRESH_MEAN_C,  cv2.THRESH_BINARY, 11, 5)
    

    #Morphological Transformations
    kernel = np.ones((3, 3), np.uint8)
    res = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel)

    res=cv2.fastNlMeansDenoising(img, dst=None, h=10, templateWindowSize=7, searchWindowSize=21)

    filename = os.path.splitext(os.path.basename(img_path))[0]
    save_path = os.path.join(output_path, f"{filename}_adaptive.jpg")
    ext = os.path.splitext(save_path)[1]
    result, encoded_img = cv2.imencode(ext, res)
    with open(save_path, mode='wb') as file:
        file.write(encoded_img)

img_path=r'C:\Users\Minjeong\Desktop\파기딥프젝\example.jpg'
output_path=r'C:\Users\Minjeong\Desktop\파기딥프젝'
otsu1(img_path, output_path)


In [ ]:
folder_path=r'C:\Users\User\HTP_Test\unzipped_data\train\image'
output_path=r'C:\Users\User\HTP_Test\YOLO\train\images'
#otsu(folder_path, output_path)
len(os.listdir(r'C:\Users\User\HTP_Test\YOLO\train\images'))

100%|██████████| 44800/44800 [10:46<00:00, 69.26it/s]


44800

In [ ]:
folder_path=r'C:\Users\User\HTP_Test\unzipped_data\validation\image'
output_path=r'C:\Users\User\HTP_Test\YOLO\valid\images'
#otsu(folder_path, output_path)
len(os.listdir(r'C:\Users\User\HTP_Test\YOLO\valid\images'))

100%|██████████| 5600/5600 [01:45<00:00, 53.00it/s]


5600